# Graph Autoencoders

## Autoencoder Networks

<b>Autoencoder</b> networks are designed to encode the date in a form of a <b>latent variables</b> vector and decode it in a form as close to the original data as possible:

[![](http://kvfrans.com/content/images/2016/08/autoenc.jpg)](http://kvfrans.com/variational-autoencoders-explained/)

We are training two networks at the same time - <b>encoder</b>), which maps data into a latent variables space $f: \mathbb{R}^d \to \mathbb{R}^k$, $z = f(x)$ and <b>decoder</b>, which reconstructs latents into the input space $\tilde{x} = g(z)$

Autoencoders have wide array of applications:

- They are effective compression tool.
- They might be used as an encryption method. 
- They are used to denoise data
- They are able to learn the non-trivial relationships between data.
- They are a perfect tool for dimensionality reduction and data embedding.
- They could be used as a generative model.


Why autoencoders are so effective for the task of dimensionality reduction?

According to <b>Manifold hypothesis</b> the local relations between data are shaped by the low-dimensional manifolds embedded within the high-dimensional space:

[![](https://www.researchgate.net/profile/Y_Bengio/publication/221700451/figure/fig1/AS:305523212734474@1449853818623/Example-of-a-simple-manifold-in-the-space-of-images-associated-with-a-rather-low-level.png)](https://www.researchgate.net/figure/Example-of-a-simple-manifold-in-the-space-of-images-associated-with-a-rather-low-level_fig1_221700451/)



## Graph Autoencoders

Before starting we must understand how <b>Graph Autoencoders (GAE) </b> differ from the regular ones.

The first big problem is the way how we should handle the data. For a given graph $g = G(V, \mathcal{E})$ and some feature vector $X$ we must define a propagation rule. 

In the simpliest form it might look like that:
$$H^l = f(AH^{l-i}\theta_{i})$$
where $A$ is adjacenct matrix, $H^l$ output on $l$th layer (clearly $H^0 = X$), $f$ is a non-linear activation function (e.g. ReLU or sigmoind) and $\theta_i$ is a matrix of weights on $i$th layer.

The problem is that with such a simple rule, node is represented as a sum of its neighbors features; </b>it is not including its own features!</b>, so we must replace $A$ with:
$$ \hat{A} = A + I $$ 

However, we still face some important issue: <b>input is not normalized</b>; nodes with large degrees will have large values in their feature representation while nodes with small degrees will have small values. Obviously, as in all the other cases it will cause many problems during the training process, making it basically impossible to learn a proper features of graph. We could normalize the input by multiplying it with the inverse degree matrix $D$:
$$H^l  = f(D^{-1}\hat{A}H^{l-i}\theta_i) $$ 

But we could further improve it. Right now $i$th node representation is impacted only by its own degree, we could rewrite the rule in a way, that includes also the degree of its neighbors:

$$H^l  = f(D^{-0.5}\hat{A}D^{-0.5}H^{l-i}\theta_i) $$ 

Such form is known as <b>Graph Convolutional Network (GCN)</b> 

## Model

We will use [flux.jl](http://fluxml.ai/):

- [Flux](http://fluxml.ai/) is a Julia machine learning stack.
- Flux is lightweight, written entirely in Julia; it is trivial to hack it and build models suited for very specific cases. 
- Flux supports all the Julia's syntax; the vast majority of Julia's functions and macros could be used inside the model.
- Building of basic models is very easy and intuitive.

Knowing that we could start building our simple model:

In [3]:
using Graphs #v1.4.1
using GraphPlot #v0.5.0
using LinearAlgebra
using Flux #v0.12.8
using Flux: @functor, @epochs, throttle
using Flux.Losses: logitbinarycrossentropy
using Zygote #v0.6.32
using CUDA #v3.5.0
using BSON #v0.3.4
using JLD2  #v0.4.15
using PyCall #v1.92.5
#Packages for saving plots to the file - they are not colored in jupyter notebook
using Cairo #v1.0.5
using Compose #v0.9.2

In [2]:
using Pkg; Pkg.status()

      Status `C:\Users\barto\.julia\environments\v1.7\Project.toml`
  [fbb218c0] BSON v0.3.4
  [052768ef] CUDA v3.5.0
  [159f3aea] Cairo v1.0.5
  [a81c6b42] Compose v0.9.2
  [587475ba] Flux v0.12.8
  [a2cc645c] GraphPlot v0.5.0
  [86223c79] Graphs v1.4.1
  [7073ff75] IJulia v1.23.2
  [033835bb] JLD2 v0.4.15
  [438e738f] PyCall v1.92.5
  [2913bbd2] StatsBase v0.33.13
  [e88e6eb3] Zygote v0.6.32


Upload the data. We will use today famous [CORA](https://relational.fit.cvut.cz/dataset/CORA) dataset:

In [4]:
@load "cora_features.jld2" features
@load "cora_graph.jld2" g
@load "cora_labels.jld2" labels

1-element Vector{Symbol}:
 :labels

and define some parameters:

In [5]:
cuda = true
num_nodes = nv(g)
num_features = size(features)[1]
hl1 = 32
hl2 = 16
epochs = 5000
β = 10.f0
λ = 0.2f0

0.2f0

Now, we could check if might use the [CUDA](https://fluxml.ai/Flux.jl/stable/gpu/) on our machine:

In [6]:
# GPU config
if cuda && CUDA.has_cuda()
    device = gpu
    @info "Training on GPU"
else
    device = cpu
    @info "Training on CPU"
end

┌ Info: Training on GPU
└ @ Main In[6]:4


### Defining Layers

Flux allows to define the layer on neural network in many different ways and it is able to handle them efficiently. We could obviously use the already implemented, most popular [types of layers](https://fluxml.ai/Flux.jl/stable/models/layers/#Basic-Layers-1) and also specific layers implemented for different kinds of models (e.g. [Graph Neural Networks](https://github.com/FluxML/GeometricFlux.jl/tree/master/src/layers).

But this time we will start with defining the layers on our own:

In [7]:
#Auxiliary function which returns normalized adjacency matrix:

𝐴̂(g) = Float32.(diagm((degree(g) .+ 1).^0.5)^-1 * (adjacency_matrix(g) + I) * diagm((degree(g) .+ 1).^0.5)^-1)

𝐴̂ (generic function with 1 method)

In [8]:
struct GraphConv{A<:AbstractMatrix, B, F}
    weight::A
    bias::B
    σ::F
    Ã::A
end

function GraphConv(g::AbstractGraph, ch::Pair{Int,Int}, σ=identity;
                 init=Flux.glorot_uniform, bias::Bool=true)
    Ã = 𝐴̂(g)
    in, out = ch
    W = init(out, in)
    b = Flux.create_bias(W, bias, out)
    GraphConv(W, b, σ, Ã)
end

GraphConv

And then, to use such user-defined layer in the Flux, we must use the macro <tt>@functor </tt>. Without that we will not be able to use all the built-in functions, e.g.  gradient propagation or [GPU computing](https://fluxml.ai/Flux.jl/stable/gpu/). 

In [9]:
@functor GraphConv

Finally, we must define which parameters of the layer are trainable and how to call it:

In [10]:
Flux.trainable(layer::GraphConv) = (layer.weight, layer.bias)

#overload call:
(layer::GraphConv)(x::AbstractMatrix) = layer.σ.(layer.weight * x * layer.Ã .+ layer.bias)

In the same way we could define the decoder part of the network:

In [11]:
struct Decoder
    σ
end

Decoder(;σ = identity) = Decoder(σ)

@functor Decoder

(dec::Decoder)(Z::AbstractMatrix)::AbstractMatrix = dec.σ.(Z'*Z)

### Stacking layers

Obviously, our model will have more than one layer. Again, there are plenty of methods to do this:

In [12]:
layer₁ = GraphConv(g, num_features=>hl1, relu)
layer₂ = GraphConv(g, hl1=>hl2)

GraphConv{Matrix{Float32}, Vector{Float32}, typeof(identity)}(Float32[0.1930431 -0.16946422 … 0.0662159 0.29370537; 0.21694158 -0.051203024 … 0.19607127 0.29335517; … ; 0.120821275 -0.34226692 … 0.22911903 0.10752668; 0.21480778 -0.3384606 … -0.16694678 0.20943451], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], identity, Float32[0.16666667 0.0 … 0.0 0.0; 0.0 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.2 0.0; 0.0 0.0 … 0.0 0.25])

Function <tt>Chain</tt> allows us to merge multiple layers in the sequence:

In [13]:
m₁ = Chain(layer₁, layer₂) 

Chain(
  GraphConv{Matrix{Float32}, Vector{Float32}, typeof(relu)}(Float32[-0.005366459 -0.02643503 … 0.028116353 0.059719276; 0.033408955 0.020218657 … -0.033923812 -0.059638724; … ; -0.008701907 -0.05799105 … -0.02755315 0.005056249; -0.034397703 -0.020249989 … 0.042670622 -0.018275753], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], NNlib.relu, Float32[0.16666667 0.0 … 0.0 0.0; 0.0 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.2 0.0; 0.0 0.0 … 0.0 0.25]),  # 45_888 parameters, plus 7_333_264
  GraphConv{Matrix{Float32}, Vector{Float32}, typeof(identity)}(Float32[0.1930431 -0.16946422 … 0.0662159 0.29370537; 0.21694158 -0.051203024 … 0.19607127 0.29335517; … ; 0.120821275 -0.34226692 … 0.22911903 0.10752668; 0.21480778 -0.3384606 … -0.16694678 0.20943451], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], identity, Float32[0.16666667 0.0 … 0.0 0.0; 0.0 0.5 … 0.0 0.0; … ; 0.0 0.0 … 0.2 0.0; 0.0 0.

We could also define the model as a function composition:

In [14]:
m₂(x) = layer₂(layer₁(x))

m₃(x) = layer₁ ∘ layer₂  

m₃ (generic function with 1 method)

or pipeline:

In [15]:
m₄(x) = layer₁(x) |> layer₂

m₄ (generic function with 1 method)

## Cost function

We could use  the one [implemented in Flux](https://github.com/FluxML/Flux.jl/blob/8f73dc6e148eedd11463571a0a8215fd87e7e05b/src/layers/stateless.jl), but in some cases we must define our own: 

This time we will work with a specific cost function defined for the graph embedding problem:

$$  \mathcal{L} = || (A - \hat{A}) \odot B ||^2_2$$

wher $\odot$ denotes the Hadamard product, and B is a matrix, such that $B_{ij} = 1$ if $A_{ij} = 0$, $b_{ij} = \beta > 1$ otherwise.
Obviously we want to ensure that our model will be able to recreate the initial adjacency matrix $A$. However, we cannot use any of the common loss functions (e.g. MSE or Log-loss) because our graph is sparse - the model will overestimate  the role of zeros in the adjacency matrix, thus it (very possibly) will learn to represent a graph as an empty matrix. To avoid that we must use the scaling matrix $B$, where every edge is weighted with $\beta > 1$. 

In [16]:
L₁(ŷ, y, β) = (b = y_train .* (β - 1.f0) .+ 1.f0; sum(((ŷ .- y) .* b).^ 2))

L₁ (generic function with 1 method)

But there is one more thing; good embedding should preserve not only the global structure of the graph. We also want to ensure that similar  vertexes are mapped closely in the embedding space. This problem is solved by adding a penalty similar to Laplacian Eigenmaps: 
$$  \mathcal{L}_2 = 2 * tr(Z^TLZ) $$
As a result, the model preserves the first-order proximity.

In [17]:
L₂(Z, L) = 2 * tr(Z * L * transpose(Z))

L₂ (generic function with 1 method)

In [18]:
## Loss
loss(x, y, L, β = β, λ = λ) = (Z = encoder(x); L₁(decoder(Z), y,  β) + λ * L₂(Z, L))

loss (generic function with 3 methods)

## Optimization

[Goodfellow I., Bengio Y., Courville A. (2016), Deep Learning, Chapter 8](http://www.deeplearningbook.org/contents/optimization.html)

Choice of the proper optimization algorithm is the most important step during the neural network training. Cost function minimization is non-trivial task; there are plenty of serious problems which might derail the learning process:
- ill-conditioned Hessian matrix.
- local minimas, plateaus, etc.
- vanishing and exploding gradients

As a result vanilla stochastic gradient method might not be able to solve such problem. There are many modifications of SGD algorithm introduced to overcome these issues:
- SGD [(Robbins & Munro 1951)](https://projecteuclid.org/download/pdf_1/euclid.aoms/1177729586)
- SGD with momentum [(Polyak, 1964)](http://www.mathnet.ru/php/archive.phtml?wshow=paper&jrnid=zvmmf&paperid=7713&option_lang=eng)
- SGD with  Nesterov momentum ([Nesterov, 1983](http://www.cis.pku.edu.cn/faculty/vision/zlin/1983-A%20Method%20of%20Solving%20a%20Convex%20Programming%20Problem%20with%20Convergence%20Rate%20O%28k%5E%28-2%29%29_Nesterov.pdf), [2005](https://www.math.ucdavis.edu/~sqma/MAT258A_Files/Nesterov-2005.pdf))
- AdaGrad (Adaptive Gradient Algorithm) [(Duchi et. al. 2011)](http://www.jmlr.org/papers/volume12/duchi11a/duchi11a.pdf)
- ADAM (Adaptive Moment Estimation) [(Kingma & Ba, 2015)](https://arxiv.org/abs/1412.6980)



Flux allows to [calculate a gradient of every function](https://fluxml.ai/Flux.jl/stable/models/basics/):

In [19]:
f(x) = 3x^2 + 2x + 1

# df/dx = 6x + 2
df(x) = gradient(f, x)[1]

df(2) # 14.0 

# d²f/dx² = 6
d²f(x) = gradient(df, x)[1]

d²f(2) # 6.0 

6.0

Even if the function is not declared as a mathematical formula:

In [20]:
function pow(x, n)
    r = 1
    for i = 1:n
        r *= x
    end
    return r
end

pow (generic function with 1 method)

In [21]:
pow(2,4)

16

In [22]:
gradient(x -> pow(x, 3), 5)

(75.0,)

In [23]:
pow2(x, n) = n <= 0 ? 1 : x*pow2(x, n-1)

pow2 (generic function with 1 method)

In [24]:
gradient(x -> pow2(x, 3), 5)

(75.0,)

It is possible because of the efficient differentiation algorithm implemented in the [<tt>Zygote.jl</tt>](https://fluxml.ai/Zygote.jl/latest/) package. It uses the characteristic elements of the language, e.g. its compiler to find the derivatives of functions. A brief explanation how <tt>Zygote</tt> works is avalaible [here](https://github.com/MikeInnes/diff-zoo) and [here](https://arxiv.org/pdf/1810.07951.pdf).

### Automatic Differentiation

The key element of the neural networks training is finding a gradients. Naïve numerical approximation of derivatives by the definition:
$$\frac{df}{dx} = \lim_{h \to 0}\frac{f(x_0 +h) - f(x_0)}{h}$$
is a recipe for disaster. So, how we could find the derivatives on the computer?

It turns out, that even the most complicated function could be represented as a composition of some basic functions (sin,cos,log,etc.) and arithmetic operations. Knowing the derivatives of these basic functions, we could compute basically every derivative by using a <i>chain rule</i>:

$$
\frac{dy}{dx} = \frac{dy_1}{dx}*\frac{dy_2}{dy_1}*\dots*\frac{dy_{n-1}}{dy_{n-2}}*\frac{dy}{dy_{n-1}}
$$

There are two distinct methods of automatic differentiation:

- <b>Forward Accumulation</b> we start with a known value of $\frac{dy_0}{dx} = \frac{dx}{dx} = 1$. then we are computing the value of the next step $\frac{dy_1}{dx} = \frac{d_1}{dx} = 1$ and we continue this process, calculating the values of next elements of chain:$\frac{dy_{i+1}}{dy_i}$, until we reach a final step.

- <b>Backward Accumulation</b> we start with a known value of  $\frac{dy}{dy_n} = \frac{dy}{dy} = 1$, then we are finding the value of: $\frac{dy}{dy_n}$, $\frac{dy}{dy_{n-1}}$, ... $\frac{dy}{dy_1}$, $\frac{dy}{dx}$.  

### Zygote.jl

<tt>Zygote</tt> package is based around two crucial elements: macro <tt>@adjoint</tt> and function <tt>pullback</tt>. 

<tt>pullback</tt> returns two things, the value of original function $y = f(x)$ and <i>pullback</i> expression $ \mathcal{B}(\overline{y}) = \overline{y}  \frac{dy}{dx}$, where $\overline{y} = \frac{dl}{dy}$ is a predefined parameter for a given function $l$.  

In [25]:
y, back = Zygote.pullback(sin, π);
y

1.2246467991473532e-16

In particular, for function <tt>gradient</tt>  $l = y = f(x)$ and  $\overline{y} = \frac{dy}{dl} = 1$:

In [26]:
back(1)

(-1.0,)

In [27]:
gradient(sin,π) == back(1)

true

Macro <tt>@adjoint</tt> allows us to compute custom adjoints and modify the derivation mechanism:

In [28]:
using Zygote: @adjoint

minus(a,b) = a - b
gradient(minus,2,3)

(1.0, -1.0)

In [29]:
minus2(a,b) = a - b
@adjoint minus2(a,b) = minus2(a,b), c̄ -> (nothing, -b^2)
gradient(minus2,2,3)

(nothing, -9.0)

 Obviously Flux have implemented [the most common optimization algorithms](https://fluxml.ai/Flux.jl/stable/training/optimisers/).

## Wrapping things up

After learning the basic mechanism of the package, we could return to our example and build a model:

In [30]:
## train data

X_train = Matrix{Float32}(features) |> device
y_train = Matrix{Float32}(adjacency_matrix(g)) |> device
L_train = Matrix{Float32}(laplacian_matrix(g)) |> device

## Model
encoder = Chain(GraphConv(g, num_features=>hl1, relu),
                GraphConv(g, hl1=>hl2)) |> device;
decoder = Decoder() |> device
model = Chain(encoder,decoder) |> device;

In [31]:
loss(X_train, y_train, L_train)

881437.2f0

In [32]:
# Training
ps = Flux.params(encoder, decoder)
train_data = [(X_train, y_train, L_train, β, λ)]
opt = ADAM(0.01)

ADAM(0.01, (0.9, 0.999), IdDict{Any, Any}())

Again, there are many methods of controlling the training process. We could define a loop and hadle the gradients on our own:

In [ ]:
function my_custom_train!(loss, ps, data, opt)
  # training_loss is declared local so it will be available for logging outside the gradient calculation.
  local training_loss
  ps = Params(ps)
  for d in data
    gs = gradient(ps) do
      training_loss = loss(d...)
      # Code inserted here will be differentiated, unless you need that gradient information
      # it is better to do the work outside this block.
      return training_loss
    end
    # Insert whatever code you want here that needs training_loss, e.g. logging.
    # logging_callback(training_loss)
    # Insert what ever code you want here that needs gradient.
    # E.g. logging with TensorBoardLogger.jl as histogram so you can see if it is becoming huge.
    update!(opt, ps, gs)
    # Here you might like to check validation set accuracy, and break out to do early stopping.
  end
end

or Flux can control the entire learning process with a function <tt>train!</tt>:

In [ ]:
#Flux.train!(loss, ps, train_data, opt)

However, <tt>train!</tt> works for only one epoch. If we need to train the model for more than one epoch, we must either modify the dataset:

In [ ]:
using Base.Iterators: repeated
dataset = repeated((x, y), 200)

or use the macro <tt>@epochs</tt>:

In [ ]:
@epochs epochs Flux.train!(loss, ps, train_data, opt)

In flux we could also define the callbacks, which will help us control the learning process:

In [ ]:
evalcb() = @show(loss(X_train, y_train))

This time we will go with a loop:

In [33]:
@info("Beginning training loop...")
lowest_loss = Inf 
last_improvement = 0
for epoch = 1:epochs
    Flux.train!(loss, ps, train_data, opt)
    loss_val = loss(X_train, y_train, L_train)
    @info("Epoch $epoch: loss value: $loss_val")
    if loss_val ≤ lowest_loss
        @info(" -> New best score! Saving model out to GAE.bson")
        BSON.@save "GAE.bson" encoder decoder epoch loss_val
        lowest_loss = loss_val
        last_improvement = epoch
    end
    if epoch - last_improvement >= 5 && opt.eta > 1e-20
        opt.eta /= 10.0
        @warn(" -> Haven't improved in a while, dropping learning rate to $(opt.eta)!")
        last_improvement = epoch
    end
    if epoch - last_improvement >= 10
        @warn(" -> We're calling this converged.")
        break
    end
end

┌ Info: Beginning training loop...
└ @ Main In[33]:1
┌ Info: Epoch 1: loss value: 3.6795936e7
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 2: loss value: 865350.75
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3: loss value: 879518.0
└ @ Main In[33]:7
┌ Info: Epoch 4: loss value: 876658.2
└ @ Main In[33]:7
┌ Info: Epoch 5: loss value: 1.0115753e6
└ @ Main In[33]:7
┌ Info: Epoch 6: loss value: 929083.06
└ @ Main In[33]:7
┌ Info: Epoch 7: loss value: 864450.94
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 8: loss value: 848491.2
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 9: loss value: 844863.1
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 10: loss value: 843620.2
└ @ Main In[33]:7
┌ 

┌ Info: Epoch 76: loss value: 673466.3
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 77: loss value: 672728.56
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 78: loss value: 672005.2
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 79: loss value: 671307.06
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 80: loss value: 670596.9
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 81: loss value: 669853.06
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 82: loss value: 669058.56
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 83: loss value: 668182.6
└ @ Main In[33]:7
┌ Info:  -> New

┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 150: loss value: 544970.94
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 151: loss value: 544575.7
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 152: loss value: 544190.94
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 153: loss value: 543826.7
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 154: loss value: 543458.7
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 155: loss value: 543114.4
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 156: loss value: 542774.8
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main I

┌ Info: Epoch 222: loss value: 532062.6
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 223: loss value: 531962.06
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 224: loss value: 531859.6
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 225: loss value: 531757.06
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 226: loss value: 531654.75
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 227: loss value: 531553.1
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 228: loss value: 531452.5
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 229: loss value: 531353.1
└ @ Main In[33]:7
┌ Info: 

┌ Info: Epoch 304: loss value: 526783.0
└ @ Main In[33]:7
┌ Info: Epoch 305: loss value: 526814.6
└ @ Main In[33]:7
┌ Info: Epoch 306: loss value: 526803.4
└ @ Main In[33]:7
┌ Info: Epoch 307: loss value: 526758.5
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 308: loss value: 526727.7
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 309: loss value: 526734.2
└ @ Main In[33]:7
┌ Info: Epoch 310: loss value: 526752.0
└ @ Main In[33]:7
┌ Info: Epoch 311: loss value: 526745.6
└ @ Main In[33]:7
┌ Info: Epoch 312: loss value: 526716.5
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 313: loss value: 526694.25
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 314: loss value: 526694.5
└ @ Main In[33]:7
┌ Info: Epoch 315: loss value: 526702.4
└ @ Main In[33]:7
┌ I

...

┌ Info: Epoch 3341: loss value: 462599.53
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3342: loss value: 462595.44
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3343: loss value: 462591.56
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3344: loss value: 462586.16
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3345: loss value: 462580.88
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3346: loss value: 462573.5
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3347: loss value: 462565.22
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3348: loss value: 462556.88
└ @ Main In[33

┌ Info: Epoch 3413: loss value: 462178.12
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3414: loss value: 462171.84
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3415: loss value: 462165.06
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3416: loss value: 462157.28
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3417: loss value: 462149.34
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3418: loss value: 462141.25
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3419: loss value: 462133.72
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3420: loss value: 462126.9
└ @ Main In[33

┌ Info: Epoch 3485: loss value: 461733.47
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3486: loss value: 461729.1
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3487: loss value: 461724.53
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3488: loss value: 461717.7
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3489: loss value: 461709.84
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3490: loss value: 461700.75
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3491: loss value: 461692.25
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3492: loss value: 461685.5
└ @ Main In[33]:

┌ Info: Epoch 3557: loss value: 461321.34
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3558: loss value: 461315.12
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3559: loss value: 461308.47
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3560: loss value: 461301.75
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3561: loss value: 461295.75
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3562: loss value: 461290.53
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3563: loss value: 461285.8
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3564: loss value: 461281.47
└ @ Main In[33

Excessive output truncated after 524304 bytes.┌ Info: Epoch 3629: loss value: 460974.06
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3630: loss value: 460968.56
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3631: loss value: 460962.1
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3632: loss value: 460954.97
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3633: loss value: 460948.34
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3634: loss value: 460942.72
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Epoch 3635: loss value: 460938.16
└ @ Main In[33]:7
┌ Info:  -> New best score! Saving model out to GAE.bson
└ @ Main In[33]:9
┌ Info: Ep

We could load the results using [BSON](https://github.com/JuliaIO/BSON.jl) file (we could obviously use other formats, but this one is particulary popular among the Julia users) and test them:

In [52]:
BSON.@load "GAE.bson" encoder decoder epoch loss_val

We will compare it with Laplacian Eigenmaps:

In [35]:
## Laplacian eigenmaps embedding
function LE(g, dim=2)
    L_norm = diagm((degree(g)).^0.5)^-1 * Matrix{Float32}(laplacian_matrix(g)) * diagm((degree(g)).^0.5)^-1
    w, v = eigen(L_norm)
    return real.(v[:, 2:dim+1])
end

LE (generic function with 2 methods)

and use [UMAP](https://umap-learn.readthedocs.io/en/latest/) to plot results:

In [36]:
umap = pyimport("umap")

PyObject <module 'umap' from 'C:\\Users\\barto\\anaconda3\\lib\\site-packages\\umap\\__init__.py'>

Let start with plot of original graph:

In [37]:
comm_color = ["red",  "green", "blue",
              "black", "yellow",  "orange", "purple"];

In [53]:
#gplot(g, nodefillc=comm_color[Flux.onecold(labels)])
# save to svg
draw(SVG("g1.svg", 16cm, 16cm), gplot(g, nodefillc=comm_color[Flux.onecold(labels)]))

false

![](g1.svg "graph")

and LEM embedding:

In [39]:
em = LE(g, 16);

In [40]:
Y = umap.UMAP().fit_transform(em);

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall C:\Users\barto\.julia\packages\PyCall\3fwVL\src\numpy.jl:67


In [41]:
#gplot(g, Y[:, 1], Y[:, 2],nodefillc=comm_color[Flux.onecold(labels)])
draw(SVG("g2.svg", 16cm, 16cm),gplot(g, Y[:, 1], Y[:, 2],nodefillc=comm_color[Flux.onecold(labels)]))

false

![](g2.svg "LEM embedding")

Finally, lets look at Autoencoder embedding:

In [54]:
Y = umap.UMAP().fit_transform(cpu(encoder(X_train))');

In [56]:
gplot(g, Y[:, 1], Y[:, 2],nodefillc=comm_color[Flux.onecold(labels)])
draw(SVG("g3.svg", 16cm, 16cm),gplot(g, Y[:, 1], Y[:, 2],nodefillc=comm_color[Flux.onecold(labels)]))

false

![](g3.svg "GCN embedding")